# Notebook SLTP (Setup, Load, Train, Predict)

Ce notebook illustre un flux d'exemple pour la formation rapide d'un modèle de langage en 4 étapes :

1. **S (Setup)** : Installation et import des librairies.
2. **L (Load)** : Chargement d'un dataset et d'un modèle pré-entraîné.
3. **T (Train)** : Fine-tuning du modèle sur un mini-exemple.
4. **P (Predict)** : Génération de texte avec le modèle fine-tuné.

L'objectif est d'être simple et clair pour démarrer rapidement avec la librairie [Hugging Face Transformers](https://github.com/huggingface/transformers).

---
## Avertissements
- Les paramètres d'entraînement sont volontairement petits pour éviter un coût en temps/mémoire trop élevé.
- Dans un vrai projet, vous devez utiliser un plus grand dataset et ajuster ces paramètres.
- Il est conseillé d'utiliser un **GPU** (local ou cloud) pour accélérer l'entraînement, surtout si vous augmentez la taille des données.

Allons-y !

## 1. S - Setup

Dans cette première étape, nous allons :
1. Vérifier la version de Python.
2. Installer si besoin les dépendances (Transformers, Datasets, Torch).
3. Importer les modules nécessaires.

---

In [ ]:
import sys
print(f"Version de Python: {sys.version}")

# Si besoin, décommentez pour installer les librairies :
# %pip install --upgrade pip
# %pip install transformers datasets torch

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset

print("Librairies importées avec succès !")

## 2. L - Load

Dans cette seconde étape, nous allons :
1. Charger un dataset d’exemple (ici, [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity)).
2. Sélectionner un modèle déjà pré-entraîné (ex: `distilgpt2`).
3. Préparer le tokenizer et le modèle.

Pour réduire la durée de l'exemple, on n'utilise qu'une petite portion du dataset (1%).

---

In [ ]:
# 1) Chargement du dataset (1% pour la démo)
dataset = load_dataset("yelp_polarity", split="train[:1%]")

# 2) Choix du modèle (GPT-2 "mini" = distilgpt2)
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("Dataset et modèle chargés !")
print(f"Exemple d'élément du dataset: {dataset[0]}")

## 3. T - Train

Dans cette étape, nous allons :
1. Tokenizer nos données.
2. Préparer un **Trainer** de la librairie Transformers.
3. Lancer l'entraînement (fine-tuning).

Nous utilisons un batch_size très petit et **1** époque pour accélérer la démonstration.

---

In [ ]:
# 1) Tokenisation
def tokenize_function(example):
    return tokenizer(
        example["text"], 
        truncation=True, 
        padding="max_length", 
        max_length=64
    )

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# On supprime les colonnes inutiles
tokenized_dataset = tokenized_dataset.remove_columns(["text", "label"])

# 2) Paramètres d'entraînement
training_args = TrainingArguments(
    output_dir="distilgpt2-sltp-model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    save_steps=50,
    logging_steps=10
)

# 3) Création du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

print("Début de l'entraînement...")
trainer.train()
print("Entraînement terminé !")

## 4. P - Predict (Génération)

Maintenant que le modèle est entraîné, nous allons :
1. Passer le modèle en mode évaluation.
2. Lui donner un prompt d’exemple.
3. Générer un texte et l’afficher.

---

In [ ]:
# Passage en mode évaluation
model.eval()

# Prompt de test
prompt = "This restaurant is amazing because"
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Génération
with torch.no_grad():
    output_ids = model.generate(
        input_ids,
        max_length=30,
        num_return_sequences=1,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

# Décodage du texte généré
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("----- Prompt -----")
print(prompt)
print("\n----- Generated Text -----")
print(generated_text)

# Conclusion

Nous avons complété le flux **S-L-T-P** :
- **Setup** : importation et installation des librairies.
- **Load** : chargement du dataset et du modèle.
- **Train** : fine-tuning rapide.
- **Predict** : génération d'un texte avec le modèle.

Pour aller plus loin :
- Utiliser un GPU, un dataset plus large et augmenter le nombre d’époques.
- Essayer d’autres modèles (GPT-2 complet, Bloom, Falcon, etc.).
- Ajuster la **max_length**, le **batch_size**, le **learning_rate**, etc.
- Explorer la [documentation Hugging Face](https://huggingface.co/docs) pour plus d’options (ex: DistilBERT pour la classification).

Bonne exploration !